<a href="https://colab.research.google.com/github/gledishoxha1/bookstore-web-scraping-api/blob/main/scraping_api_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "requests", "beautifulsoup4", "pandas"])

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://books.toscrape.com/"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

books = []

for book in soup.select(".product_pod"):
    title = book.h3.a["title"]
    price = book.select_one(".price_color").text.replace("Â", "")
    availability = book.select_one(".availability").text.strip()
    books.append({
        "title": title,
        "price": price,
        "availability": availability
    })

books_df = pd.DataFrame(books)

def get_book_info(title):
    search_url = "https://openlibrary.org/search.json"
    params = {"title": title, "limit": 1}
    search_data = requests.get(search_url, params=params).json()
    if not search_data["docs"]:
        return {
            "author": None,
            "publish_year": None,
            "edition_count": None
        }
    book = search_data["docs"][0]
    return {
        "author": ", ".join(book.get("author_name", [])),
        "publish_year": book.get("first_publish_year"),
        "edition_count": book.get("edition_count")
    }

api_data = [get_book_info(title) for title in books_df["title"]]

final_df = pd.concat([books_df, pd.DataFrame(api_data)], axis=1)

final_df.to_csv("final_data.csv", index=False)
final_df.to_json("final_data.json", orient="records", indent=4)

from google.colab import files

files.download("final_data.csv")
files.download("final_data.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>